# **Aprendizado Supervisionado - Segmentação Semântica**

Importando as Bibliotecas Necessárias





In [1]:
!pip3 install -U segmentation-models

In [2]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/ 

In [1]:
%env SM_FRAMEWORK=tf.keras

import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras


2023-02-18 00:53:40.298084: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 00:53:40.465268: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-18 00:53:41.409497: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/pericles/miniconda3/envs/tf2/lib/
2023-02-18 00:53:41.409581: W tensorflow/compi

Segmentation Models: using `tf.keras` framework.


In [2]:
%env SM_FRAMEWORK=tf.keras

from segmentation_models.utils import set_trainable
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import Adam, SGD
#from model import *
#from data import *
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt
import cv2
import time
from keras.callbacks import ModelCheckpoint, CSVLogger

env: SM_FRAMEWORK=tf.keras


In [3]:
def adjust_data(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask >= 0.5] = 1
        mask[mask < 0.5] = 0
    return (img,mask)

In [4]:
width_size = 512
height_size = 512

In [5]:
def train_generator(batch_size, 
                   train_path,
                   image_folder,
                   mask_folder,
                   aug_dict,
                   image_color_mode = "grayscale",
                   mask_color_mode = "grayscale", 
                   image_save_prefix  = "image", 
                   mask_save_prefix  = "mask",
                   flag_multi_class = False,
                   num_class = 1,
                   save_to_dir = None,
                   target_size = (width_size,height_size),
                   seed = 1):
    '''
    Can generate image and mask at the same time. Use the same seed for image_datagen and mask_datagen to ensure 
    the transformation for image and mask is the same. If you want to visualize the results of generator, set 
    save_to_dir = "your path"
    '''
    
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow_from_directory(train_path,
                                                        classes = [image_folder],
                                                        class_mode = None,
                                                        color_mode = image_color_mode,
                                                        target_size = target_size,
                                                        batch_size = batch_size,
                                                        save_to_dir = save_to_dir,
                                                        save_prefix  = image_save_prefix,
                                                        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(train_path,
                                                        classes = [mask_folder],
                                                        class_mode = None,
                                                        color_mode = mask_color_mode,
                                                        target_size = target_size,
                                                        batch_size = batch_size,
                                                        save_to_dir = None,
                                                        save_prefix  = mask_save_prefix,
                                                        seed = seed)
    
    train_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in train_generator:
        img,mask = adjust_data(img, mask, flag_multi_class, num_class)
        yield (img,mask)

In [6]:
def test_generator(batch_size,
                  test_path,
                  image_folder,
                  mask_folder,
                  image_color_mode = "grayscale",
                  mask_color_mode = "grayscale",
                  image_save_prefix  = "image",
                  mask_save_prefix  = "mask",
                  flag_multi_class = False,
                  num_class = 1, 
                  save_to_dir = None,
                  target_size = (width_size,height_size), seed = 2):
    '''
    Can generate image and mask at the same time. Use the same seed for image_datagen and mask_datagen to ensure 
    the transformation for image and mask is the same. If you want to visualize the results of generator, set 
    save_to_dir = "your path"
    '''
    
    image_datagen = ImageDataGenerator()
    mask_datagen = ImageDataGenerator()
    
    image_generator = image_datagen.flow_from_directory(test_path,
                                                        classes = [image_folder],
                                                        class_mode = None,
                                                        color_mode = image_color_mode,
                                                        target_size = target_size,
                                                        batch_size = batch_size,
                                                        save_to_dir = save_to_dir,
                                                        save_prefix  = image_save_prefix,
                                                        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(test_path,
                                                      classes = [mask_folder],
                                                      class_mode = None,
                                                      color_mode = mask_color_mode,
                                                      target_size = target_size,
                                                      batch_size = batch_size,
                                                      save_to_dir = save_to_dir,
                                                      save_prefix  = mask_save_prefix,
                                                      seed = seed)
    
    test_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in test_generator:
        img,mask = adjust_data(img,mask,flag_multi_class,num_class)
        yield (img,mask)

Definindo como o Modelo fará o Data Augmentation

In [7]:
data_gen_args = dict(rotation_range=180,
                    width_shift_range=0.07,
                    height_shift_range=0.07,
                    shear_range=0.07,
                    zoom_range=0.07,
                    horizontal_flip=True,
                    fill_mode='nearest')

Definindo o caminho das pastas de validação e treino. Inicializando o dimensionamento das imagens que serão lidas

In [8]:
path_dataset = '../dataset_insulators/'
train_data_path = '../dataset_insulators/train/images'
test_data_path = '../dataset_insulators/validation/images/'

num_of_imgs_train = sum(len(files) for _, _, files in os.walk(train_data_path))
num_of_imgs_test = sum(len(files) for _, _, files in os.walk(test_data_path))

num_of_train_samples = int(num_of_imgs_train)
num_of_validation_samples  = int(num_of_imgs_test)
print("Train Images : ", num_of_train_samples)
print("Validation Images  : ", num_of_validation_samples)

batch_size = 2
train_samples = num_of_train_samples
test_samples = num_of_validation_samples
steps_p_epoch = int(train_samples / batch_size)
val_steps = int(test_samples / batch_size)
nr_epochs = 5

Train Images :  34830
Validation Images  :  8932


# Colocando a rede para treinar

In [9]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


2023-02-18 00:53:57.410205: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-18 00:53:57.467009: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-18 00:53:57.467187: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [10]:
#3227
model_Unet_resnet34 = sm.Unet(backbone_name='resnet34', encoder_weights='imagenet', encoder_freeze=True, classes=1, activation='sigmoid')

#1886
model_Linknet_resnet34 = sm.Linknet(backbone_name='resnet34', encoder_weights='imagenet', encoder_freeze=True, classes=1, activation='sigmoid')

#2921.6
model_Unet_vgg16 = sm.Unet(backbone_name='vgg16', encoder_weights='imagenet', encoder_freeze=True, classes=1, activation='sigmoid')

#2897.14 Segundos medios por epoca
model_Linknet_vgg16 = sm.Linknet(backbone_name='vgg16', encoder_weights='imagenet', encoder_freeze=True, classes=1, activation='sigmoid')

all_models = [model_Unet_resnet34, model_Linknet_resnet34, model_Unet_vgg16, model_Linknet_vgg16]
all_names_models = ["model_Unet_resnet34", "model_Linknet_resnet34", "model_Unet_vgg16", "model_Linknet_vgg16"]


2023-02-18 00:54:06.779228: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 00:54:06.781708: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-18 00:54:06.781995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-18 00:54:06.782116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there

In [11]:
for index in range(len(all_names_models)):
    since = time.time()
    BACKBONE = all_names_models[index].split('_')[-1]
    DECODER = all_names_models[index].split('_')[-2]
    
    preprocess_input = sm.get_preprocessing(BACKBONE)

    trainGene = train_generator(batch_size, 
                               path_dataset, 
                               'train',
                               'mask_train',
                               data_gen_args,
                               save_to_dir = None, 
                               image_color_mode = "rgb"
                              )

    testGene = test_generator(batch_size, 
                              path_dataset,
                              'validation', 
                              'mask_validation', 
                              save_to_dir = None,
                              image_color_mode = "rgb")
    
    model = all_models[index]
    opt = Adam(learning_rate=1e-3)

    model.compile(opt,
                  loss=sm.losses.jaccard_loss,
                  metrics=[sm.metrics.IOUScore(threshold=0.5),
                  sm.metrics.FScore(threshold=0.5), 
                  sm.metrics.precision])

    csv_logger = CSVLogger(BACKBONE + "_" + DECODER + ".csv", append=True)
    
    model_checkpoint = ModelCheckpoint(BACKBONE + "_" + DECODER + ".hdf5", 
                                       monitor='loss',
                                       verbose=1,
                                       save_best_only=True)
    
    history = model.fit(trainGene, 
                        validation_data = testGene,
                        validation_steps = val_steps,
                        steps_per_epoch = steps_p_epoch,
                        epochs=nr_epochs,
                        callbacks=[model_checkpoint, csv_logger])
    
    time_elapsed = time.time() - since
    print(
        f"\n Total time: {round(time_elapsed // 60, 2)} minutes and {round(time_elapsed % 60, 2)} seconds"
    )


Found 34830 images belonging to 1 classes.
Found 34830 images belonging to 1 classes.
Epoch 1/5


2023-02-18 00:54:40.428574: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-02-18 00:54:41.489534: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-18 00:54:41.490487: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-18 00:54:41.490520: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:85] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-18 00:54:41.491099: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-18 00:54:41.491162: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-02-18 00:54:42.698903:

17415/17415 [==============================] - ETA: 0s - loss: 0.1010 - iou_score: 0.9001 - f1-score: 0.9432 - precision: 0.9430Found 8932 images belonging to 1 classes.
Found 8932 images belonging to 1 classes.

Epoch 1: loss improved from inf to 0.10097, saving model to resnet34_Unet.hdf5
17415/17415 [==============================] - 3313s 189ms/step - loss: 0.1010 - iou_score: 0.9001 - f1-score: 0.9432 - precision: 0.9430 - val_loss: 0.0540 - val_iou_score: 0.9463 - val_f1-score: 0.9716 - val_precision: 0.9858
Epoch 2/5
17415/17415 [==============================] - ETA: 0s - loss: 0.0540 - iou_score: 0.9462 - f1-score: 0.9713 - precision: 0.9723
Epoch 2: loss improved from 0.10097 to 0.05396, saving model to resnet34_Unet.hdf5
17415/17415 [==============================] - 3205s 184ms/step - loss: 0.0540 - iou_score: 0.9462 - f1-score: 0.9713 - precision: 0.9723 - val_loss: 0.0514 - val_iou_score: 0.9488 - val_f1-score: 0.9728 - val_precision: 0.9855
Epoch 3/5
17415/17415 [=======

Epoch 3/5
17415/17415 [==============================] - ETA: 0s - loss: 0.0523 - iou_score: 0.9480 - f1-score: 0.9728 - precision: 0.9736
Epoch 3: loss improved from 0.05951 to 0.05229, saving model to vgg16_Linknet.hdf5
17415/17415 [==============================] - 2969s 170ms/step - loss: 0.0523 - iou_score: 0.9480 - f1-score: 0.9728 - precision: 0.9736 - val_loss: 0.0440 - val_iou_score: 0.9562 - val_f1-score: 0.9773 - val_precision: 0.9892
Epoch 4/5
15739/17415 [==========================>...] - ETA: 4:09 - loss: 0.0486 - iou_score: 0.9516 - f1-score: 0.9748 - precision: 0.9753

KeyboardInterrupt: 

In [2]:
import splitfolders

def create_default_dataset(path_to_folder: str, path_to_save: str, percent_of_images: list = [0.98, 0.02, 0.0]):
    """Create a dataset with the structure: train, test, validation. """
    train_percent = percent_of_images[0]
    test_percet = percent_of_images[1]
    val_percent = percent_of_images[2]

    splitfolders.ratio(f"{path_to_folder}",
                   output=f"{path_to_save}",
                   seed=42,
                   ratio=(train_percent, val_percent, test_percet),
                   group_prefix=True,
                   move=False
   )

In [3]:
create_default_dataset(path_to_folder='../dirty_insulators/', path_to_save='new_data/')

Copying files: 14424 files [00:23, 603.70 files/s] 
